# This notebook is a tutorial on NetDebugger
Author: Rishi Gurnani, Georgia Institute of Technology<br />
Creation Date: July 21, 2021 4:54 PM EST

In [17]:
!jupyter notebook --version
!python --version
import tensorflow as tf
print(tf.__version__)
#import sklearn
#print(sklearn.__version__)
import keras
print(keras.__version__)
import pandas as pd 
print(pd.__version__)
# 6.4.0
# Python 3.8.10
# 2.2.0
# 2.4.3
# 1.2.4
###################################################################
#https://nanohub.org/tools/jupyter70
#Jupyter Notebook with Anaconda 2020.11
#6.4.0, Python 3.8.10, tf 2.2.0, sklearn 1.0, keras 2.4.3 pd 1.2.4
###################################################################

6.4.0
Python 3.8.10
2.2.0
2.4.3
1.2.4


In [18]:
!pip list
#!pip freeze

Package                            Version
---------------------------------- -------------------
absl-py                            0.12.0
aiohttp                            3.7.4
alabaster                          0.7.12
anaconda-client                    1.7.2
anaconda-navigator                 2.0.3
anaconda-project                   0.10.0
ansiwrap                           0.8.4
anyio                              3.1.0
appdirs                            1.4.4
appmode                            0.8.0
argh                               0.26.2
argon2-cffi                        20.1.0
arrow                              1.1.0
ase                                3.21.1
asn1crypto                         1.4.0
astor                              0.8.1
astroid                            2.5.7
astropy                            4.2.1
astunparse                         1.6.3
async-generator                    1.10
async-timeout                      3.0.1
atomicwrites                       1

# Import
Some python packages are needed to run this notebook. We import all of those below.

In [19]:
# standard libraries
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from torch import tensor, cuda, manual_seed, zeros, nn, optim
from torch import float as torch_float
from torch_geometric.data import DataLoader
from torch import device as torch_device
import random
from sklearn.model_selection import train_test_split
from torch_geometric.data import Data

# nndebugger functions
from bin.nndebugger import constants, loss, dl_debug
from bin.nndebugger import torch_utils as utils

# Fix random seeds to ensure reproducible results

In [20]:
random.seed(constants.RANDOM_SEED)
manual_seed(constants.RANDOM_SEED)
np.random.seed(constants.RANDOM_SEED)

# Load data set 

In [21]:
data_df = pd.read_csv('data/export.csv',index_col=0)
data_df.head()

,smiles,property,value
822,[*]C[*],Egc,6.8972
823,[*]CC([*])C,Egc,6.5196
824,[*]CC([*])CC,Egc,6.5170
825,[*]CC([*])CCC,Egc,6.7336
826,[*]CC([*])CC(C)C,Egc,6.7394


In [22]:
len(data_df)

3380

# Featurize data set

In [23]:
N_FEATURES = 512
N_DATA = len(data_df)

def featurize_smiles(smile):
    smile = smile.replace('*', 'H')
    mol = Chem.MolFromSmiles(smile)
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=N_FEATURES, useChirality=True)
    return np.array(fp)

feature_array = np.zeros((N_DATA, N_FEATURES))

ind = 0
for smiles in data_df.smiles.values:
    feature_array[ind,:] = featurize_smiles(smiles)
    ind += 1

# Write a logical architecture that will pass all test cases

In [24]:
class MyNet(nn.Module):
    
    def __init__(self, input_dim, output_dim, capacity): # output_dim as only one property bandgap we looking

        super(MyNet,self).__init__()
        self.layers = nn.ModuleList()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.n_hidden = capacity
        unit_sequence = utils.unit_sequence(self.input_dim, 
                                            self.output_dim, 
                                            self.n_hidden)
        self.relu = nn.ReLU()
        # set up hidden layers
        for ind,n_units in enumerate(unit_sequence[:-2]):
            size_out_ = unit_sequence[ind+1]
            layer = nn.Linear(n_units, size_out_)
            self.layers.append(layer)

        # set up output layer
        size_in_ = unit_sequence[-2]
        size_out_ = unit_sequence[-1]
        layer = nn.Linear(size_in_, size_out_)
        self.layers.append(layer)
    
    def forward(self, data):  # pytorch, how to forward path to occure
        x = data.x
        for i in range(len(self.layers)):  # loop over all the hidden layers, inside each layer do matric multiplication
            x = self.layers[i](x)          # matric multiplication
            if i < (self.n_hidden - 1):    # checked to see if layers and hidden layer.
                x = self.relu(x)           #  nd if it is a hidden layer,  then I will do regular non-linearity
   
        return x.view(data.num_graphs,)  #this is the return line of the view method The View method ensures that the output of the, of my net match, the shape of the labels.

# a list of models that are bug free!
capacity_ls = [1,2,3]  #   I create a list of functions called correct model class LS, the first function in the list Returns on my net of capacity one, the second returns that - of capacity 2 and the third returns a my net of capacity three.
correct_model_class_ls = [lambda : MyNet(N_FEATURES, 1, capacity) for capacity in
                          capacity_ls]

# Prepare inputs for DebugSession

In [25]:
# bug free processing pipeline! 
# just responsible for massaging. Some inputs. That will need to run that netdebugger
model_type = 'mlp'
# data_set
n_test = int(np.floor(N_DATA*constants.TRAIN_FRAC))
n_train = N_DATA - n_test
(X_train, X_test, label_train, 
label_test) = train_test_split(
                                    feature_array,
                                    data_df.value.values.tolist(),
                                    test_size=n_test,
                                    shuffle=True,
                                    random_state=constants.RANDOM_SEED
                                )

train_X = [Data(x=tensor(X_train[ind,:], dtype=torch_float).view(1,N_FEATURES),
                y=tensor(label_train[ind], dtype=torch_float)
            ) 
            for ind in range(n_train)]
data_set = train_X
zero_data_set = [Data(x=zeros((1,N_FEATURES)), y=x.y) for x in train_X]
loss_fn = loss.st_loss()
target_mean = np.mean(label_train)
device = torch_device('cuda' if cuda.is_available() else 'cpu')

# Test output shape

The shape of the model output should match the shape of the labels.

In [26]:
# this cell should pass since it uses a bug-free model
# In order to run this test. We need to instantiate what I call a net debugger, debug session
# some of our inputs, like our mynet architecture and our polymer band Gap data.
ds = dl_debug.DebugSession(correct_model_class_ls, model_type, capacity_ls, data_set, zero_data_set, loss_fn,
                 device, do_test_output_shape=True) # only do_test_output_shape is true and all other are false.
ds.main()

Training data contains 676 points


Verified that shape of model predictions is equal to shape of labels


Debug session complete. No errors detected.


In [27]:
# buggy model. Can you spot the bug?
# buggy net same as my net above only line 35 is change This is the return line of the forward method line
class BuggyNet(nn.Module):
    
    def __init__(self, input_dim, output_dim, capacity):

        super(BuggyNet,self).__init__()
        self.layers = nn.ModuleList()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.n_hidden = capacity
        unit_sequence = utils.unit_sequence(self.input_dim, 
                                            self.output_dim, 
                                            self.n_hidden)
        self.relu = nn.ReLU()
        # set up hidden layers
        for ind,n_units in enumerate(unit_sequence[:-2]):
            size_out_ = unit_sequence[ind+1]
            layer = nn.Linear(n_units, size_out_)
            self.layers.append(layer)

        # set up output layer
        size_in_ = unit_sequence[-2]
        size_out_ = unit_sequence[-1]
        layer = nn.Linear(size_in_, size_out_)
        self.layers.append(layer)
    
    def forward(self, data):
        x = data.x
        for i in range(len(self.layers)):
            x = self.layers[i](x)
            if i < (self.n_hidden - 1):
                x = self.relu(x)
   
        return x # Spoiler! The bug is here. The correct line is 'return x.view(data.num_graphs,)' 
# This is the return line of the forward method line
# a list of models that are buggy
capacity_ls = [1,2,3]
buggy_model_class_ls = [lambda : BuggyNet(N_FEATURES, 1, capacity) for capacity in
                          capacity_ls]

In [28]:
# this cell should NOT pass since it uses a buggy model 

ds = dl_debug.DebugSession(buggy_model_class_ls, model_type, capacity_ls, data_set, zero_data_set, loss_fn,
                 device, do_test_output_shape=True)
ds.main()
# pytorch, doesn't stop the training by raising an error. While not debugger, will stop the training. 

Training data contains 676 points



/apps/share64/debian10/anaconda/anaconda-7/lib/python3.8/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([6])) that is different to the input size (torch.Size([6, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


AssertionError: The model output shape torch.Size([6, 1]) and label shape torch.Size([6]) are not the same

# Test input independent baseline
The loss of the model should be lower when real features are passed in than when zeroed features are passed in.

In [29]:
# trainer without bugs!
#  input, independent Baseline, this test, make sure that the loss of the model is lower 
#when real features are passed in as opposed to when zeroed out features are passed in.

# The trainer is a function that is responsible for updating the parameters of the model based on training data.


def trainer(model, data_set, batch_size, learning_rate, n_epochs, device, loss_obj):
    
    data_loader = DataLoader(data_set, batch_size=batch_size, shuffle=True)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate) # Adam optimization
    model.train() # set model to train mode
    loss_history = []
    for epoch in range(n_epochs):  # Loop over a specified number epochs
        per_epoch_loss = 0
        for ind, data in enumerate(data_loader): # loop through training batches
            data = data.to(device) # send data to GPU, if available
            optimizer.zero_grad() # zero the gradients
            output = model(data) # perform forward pass  # within epochs line 14, performs the forwardpass
            loss = loss_obj(output, data) # compute loss  # computes the loss
            per_epoch_loss += loss.detach().cpu().numpy()
            loss.backward() # perform backward pass performs the backward pass 
            optimizer.step() # update weights and line 18 updates the parameters
        loss_history.append(per_epoch_loss) # we track with the loss has been over each epoch
    
    return loss_history   # return the loss of each at Bach



In [30]:
# this test should pass since we are using a trainer without bugs

ds = dl_debug.DebugSession(correct_model_class_ls, model_type, capacity_ls, data_set, zero_data_set, loss_fn,
                 device, do_test_input_independent_baseline=True, trainer=trainer) # do_test_input_independent_baseline=True
ds.main()

Training data contains 676 points


Checking input-independent baseline
..last epoch real_data_loss 0.22568964213132858
..last epoch zero_data_loss 14.373095512390137
Input-independent baseline is verified


Debug session complete. No errors detected.


In [31]:
# trainer with bugs! Can you spot the bug?
# buggy trainer. It is exactly the same as trainer. Except for one difference.
def buggy_trainer(model, data_set, batch_size, learning_rate, n_epochs, device, loss_obj):
    
    data_loader = DataLoader(data_set, batch_size=batch_size, shuffle=True)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate) # Adam optimization
    model.train() # set model to train mode
    loss_history = []
    for epoch in range(n_epochs):
        per_epoch_loss = 0
        for ind, data in enumerate(data_loader): # loop through training batches
            data = data.to(device) # send data to GPU, if available
            optimizer.zero_grad() # zero the gradients
            output = model(data) # perform forward pass
            loss = loss_obj(output, data) # compute loss
            per_epoch_loss += loss.detach().cpu().numpy()
            optimizer.step() # update weights
        loss_history.append(per_epoch_loss)
    
    return loss_history

# not included a line to do the backward pass, and this will definitely negatively affect model training because it means that we're not computing gradients.
# Spoiler! The bug is that there is no backward pass being performed!

In [32]:
# this test should NOT pass since we are using a buggy trainer

ds = dl_debug.DebugSession(correct_model_class_ls, model_type, capacity_ls, data_set, zero_data_set, loss_fn,
                 device, do_test_input_independent_baseline=True, trainer=buggy_trainer)
ds.main()

Training data contains 676 points


Checking input-independent baseline
..last epoch real_data_loss 134.40143203735352
..last epoch zero_data_loss 134.75465393066406


ValueError: The loss of zeroed inputs is nearly the same as the loss of
                    real inputs. This may indicate that your model is not learning anything
                    during training. Check your trainer function and your model architecture.

# Overfit small batch
If you hope to learn a good map on your whole data set using model archicture ***A***, then ***A*** should have enough capacity to completely overfit a small batch of the data set.

In [33]:
# this test should pass since we are using a good model

ds = dl_debug.DebugSession(correct_model_class_ls, model_type, capacity_ls, data_set, zero_data_set, loss_fn,
                 device, do_test_overfit_small_batch=True, trainer=trainer)
ds.main()

Training data contains 676 points


Checking if a small batch can be overfit
..Epoch 0
....Outputs -0.1179 -0.0832 -0.0910 -0.1023 -0.1179
....Labels  5.6991 5.3739 5.0497 2.9694 5.7012
....Loss: 5.164145787847168
....R2: -24.459165859291733
..Epoch 1
....Outputs -0.0351 -0.0503 -0.0165 -0.0337 -0.0503
....Labels  2.9694 5.7012 5.3739 5.0497 5.6991
....Loss: 5.1002019730011945
....R2: -23.832585224647396
..Epoch 2
....Outputs 0.0199 0.0257 0.0519 0.0095 0.0095
....Labels  5.0497 2.9694 5.3739 5.7012 5.6991
....Loss: 5.040911194917921
....R2: -23.258576547216546
..Epoch 3
....Outputs 0.0799 0.1340 0.0778 0.0778 0.0908
....Labels  5.0497 5.3739 5.6991 5.7012 2.9694
....Loss: 4.973188421236432
....R2: -22.611143571313786
..Epoch 4
....Outputs 0.2393 0.1645 0.1709 0.1514 0.1645
....Labels  5.3739 5.6991 2.9694 5.0497 5.7012
....Loss: 4.888306674583275
....R2: -21.81203734875709
..Epoch 5
....Outputs 0.2755 0.2755 0.2396 0.2719 0.3727
....Labels  5.6991 5.7012 5.0497 2.9694 5.3739
....Loss:

In [34]:
# buggy model. Can you spot the "bug"?

class BuggyNet(nn.Module):
    
    def __init__(self, input_dim, output_dim, capacity):

        super(BuggyNet,self).__init__()
        self.layers = nn.ModuleList()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.n_hidden = capacity
        unit_sequence = utils.unit_sequence(self.input_dim, 
                                            self.output_dim, 
                                            self.n_hidden)
        self.sigmoid = nn.Sigmoid() 
        # set up hidden layers
        for ind,n_units in enumerate(unit_sequence[:-2]):
            size_out_ = unit_sequence[ind+1]
            layer = nn.Linear(n_units, size_out_)
            self.layers.append(layer)

        # set up output layer
        size_in_ = unit_sequence[-2]
        size_out_ = unit_sequence[-1]
        layer = nn.Linear(size_in_, size_out_)
        self.layers.append(layer)
    
    def forward(self, data):
        x = data.x
        for i in range(len(self.layers)):
            x = self.layers[i](x)
            if i < (self.n_hidden - 1):
                x = self.sigmoid(x) # Spoiler! The "bug" is here.  my net used a relu activation function. This model uses a sigmoid activation function.
                                    # sigmoid. Activation usually results in back, gradients.
   
        return x.view(data.num_graphs,) 

# a list of models that are buggy
capacity_ls = [1,2,3]
buggy_model_class_ls = [lambda : BuggyNet(N_FEATURES, 1, capacity) for capacity in
                          capacity_ls]

In [35]:
# this test should not pass since we are using a buggy model

ds = dl_debug.DebugSession(buggy_model_class_ls, model_type, capacity_ls, data_set, zero_data_set, loss_fn,
                 device, do_test_overfit_small_batch=True, trainer=trainer)
ds.main()

Training data contains 676 points


Checking if a small batch can be overfit
..Epoch 0
....Outputs 0.0960 0.1009 0.0976 0.0959 0.0960
....Labels  5.7012 5.3739 5.0497 2.9694 5.6991
....Loss: 4.967861455966612
....R2: -22.560588584071393
..Epoch 1
....Outputs 0.4603 0.4596 0.4610 0.4603 0.4656
....Labels  5.6991 2.9694 5.0497 5.7012 5.3739
....Loss: 4.612118489684607
....R2: -19.307108964786604
..Epoch 2
....Outputs 0.8247 0.8262 0.8318 0.8262 0.8257
....Labels  2.9694 5.7012 5.3739 5.6991 5.0497
....Loss: 4.25635726278205
....R2: -16.29511017505854
..Epoch 3
....Outputs 1.1943 1.2021 1.1937 1.1962 1.1962
....Labels  5.0497 5.3739 2.9694 5.7012 5.6991
....Loss: 3.8985214071474976
....R2: -13.509319622212006
..Epoch 4
....Outputs 1.5791 1.5731 1.5694 1.5731 1.5693
....Labels  5.3739 5.7012 5.0497 5.6991 2.9694
....Loss: 3.5366167455296513
....R2: -10.940519449487216
..Epoch 5
....Outputs 1.9537 1.9596 1.9658 1.9542 1.9596
....Labels  5.0497 5.6991 5.3739 2.9694 5.7012
....Loss: 3.1690381

..Epoch 59
....Outputs 5.1997 4.9698 5.1997 4.5753 5.1280
....Labels  5.6991 5.0497 5.7012 2.9694 5.3739
....Loss: 0.7932886690368016
....R2: 0.39922802445867434
..Epoch 60
....Outputs 5.1713 5.1713 4.9361 5.0979 4.5303
....Labels  5.6991 5.7012 5.0497 5.3739 2.9694
....Loss: 0.7854485797891338
....R2: 0.41104413611789836
..Epoch 61
....Outputs 5.1465 5.0714 5.1465 4.9059 4.4883
....Labels  5.6991 5.3739 5.7012 5.0497 2.9694
....Loss: 0.7787694078111971
....R2: 0.42101816903215405
..Epoch 62
....Outputs 5.0494 4.4504 5.1263 5.1263 4.8802
....Labels  5.3739 2.9694 5.7012 5.6991 5.0497
....Loss: 0.7727887679303608
....R2: 0.4298766989373952
..Epoch 63
....Outputs 5.1115 4.4171 4.8597 5.0329 5.1115
....Labels  5.6991 2.9694 5.0497 5.3739 5.7012
....Loss: 0.7669891738152412
....R2: 0.4384018590522113
..Epoch 64
....Outputs 5.0221 5.1025 5.1025 4.8449 4.3888
....Labels  5.3739 5.7012 5.6991 5.0497 2.9694
....Loss: 0.7608916658544421
....R2: 0.44729567458000363
..Epoch 65
....Outputs 4.8359 

ValueError: Error: Your model was not able to overfit a small batch 
                               of data. The maximum R2 over 100 epochs was not greater than 0.99

# Chart Dependencies
The `forward` method should not mix information from separate instances.
Black circles are four carbon atoms and gold circles are for chlorine atoms and hydrogen's are treated implicitly

![image info](./images/graph.png)

![image info](./images/graphnet.png)

![image info](./images/graph_batch2.png)

Then we take the sum of all the added features to get a feature Vector, that represents the entire pollen repeat unit. Finally. We pass this polymer feature Vector through a number of hidden layers to get the band Gap out at the end. 

However, we must be careful when we program the normalization operation as mentioned. The features of the graph are 2D Matrix. So when we pass in a bunch of graphs together to the network, we will actually pass in a 3D Matrix. Therefore. We need to make sure that the mean is computed along this blue Dimension out of the blue might be hard to see, but along this Dimension and we need to make sure that information is not passed along. 

The red Dimension as that will mix up information of different polymers for instances. This is not desirable because the band gap of one polymer is independent of the band, gap of another polymer. 

In [36]:
# data to illustrate the point
# graph. Neural networks
# Black circles are four carbon atoms and gold circles are for chlorine atoms and hydrogen's are treated implicitly

np.random.seed(constants.RANDOM_SEED)
polymer_indices = data_df.sample(n=4).index
polymer_smiles = data_df.loc[polymer_indices, 'smiles'].values.tolist()

In [37]:
# Then we can create a 2d Matrix of features for this entire polymer by counting the types of atoms in polymer
#  where each atom contributes a row of features
# featureize  the polymers
feature_dict = {'C': np.array([1,0,0,0]),
    'O': np.array([0,1,0,0]),
    'N': np.array([0,0,1,0]),
    'Cl': np.array([0,0,0,1])
}
N_FEATURES_ = len(feature_dict)
N_DATA_ = len(polymer_smiles)
MAX_N_ATOMS = max([Chem.MolFromSmiles(smile).GetNumAtoms() for smile in polymer_smiles])
PROJECTOR_DIM = 100

def featurize_smiles_by_atom(smile):
    smile = smile.replace('*', 'H')
    mol = Chem.MolFromSmiles(smile)
    features = np.zeros((MAX_N_ATOMS, N_FEATURES_))
    for ind,atom in enumerate(mol.GetAtoms()):
        atom_feature = feature_dict[atom.GetSymbol()]
        features[ind, :] = atom_feature

    return features

labels = data_df.loc[polymer_indices, 'value'].values

train_X_ = [Data(x=tensor(featurize_smiles_by_atom(polymer_smiles[ind]), dtype=torch_float),
                    y=tensor(labels[ind], dtype=torch_float)
            ) 
            for ind in range(N_DATA_)
]

data_set_ = train_X_

In [38]:
class GraphNet(nn.Module):  # architecture graph net
    
    def __init__(self, input_dim, output_dim, capacity):

        super(GraphNet,self).__init__()
        self.layers = nn.ModuleList()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.n_hidden = capacity
        unit_sequence = utils.unit_sequence(self.input_dim, 
                                            self.output_dim, 
                                            self.n_hidden)
        self.node_projector = nn.Linear(N_FEATURES_, PROJECTOR_DIM)
        self.relu = nn.ReLU()
        # set up hidden layers
        for ind,n_units in enumerate(unit_sequence[:-2]):
            size_out_ = unit_sequence[ind+1]
            layer = nn.Linear(n_units, size_out_)
            self.layers.append(layer)

        # set up output layer
        size_in_ = unit_sequence[-2]
        size_out_ = unit_sequence[-1]
        layer = nn.Linear(size_in_, size_out_)
        self.layers.append(layer)
    
    def forward(self, data):     #forward method
        x = data.x
        x = x.view(data.num_graphs, MAX_N_ATOMS, N_FEATURES_)  #  set the dimensions of the features such that the zero with Dimension is the number of polymers. The first Dimension is related to the number of Atoms for polymer and the last Dimension is the number of features
        x = self.node_projector(x)
        x_mean = x.mean(dim=2)      #  mean to be taken over the future Dimension.
        x = x - x_mean[:, :, None] # make use of broadcasting
        x = x.sum(dim=1)
        for i in range(len(self.layers)):
            x = self.layers[i](x)
            if i < (self.n_hidden - 1):
                x = self.relu(x)
   
        return x.view(data.num_graphs,)

# a list of models that are bug free!
capacity_ls = [1,2,3]
correct_graphnet_class_ls = [lambda : GraphNet(PROJECTOR_DIM, 1, capacity) for capacity in
                          capacity_ls]

In [39]:
# this test should pass since we are using a bug-free model

ds = dl_debug.DebugSession(correct_graphnet_class_ls, 'gnn', capacity_ls, data_set_, zero_data_set, loss_fn,
                 device, do_chart_dependencies=True)
ds.main()

Training data contains 4 points


Beginning to chart dependencies
..Epoch 0
....Outputs -0.3991 -0.3810 -0.3220 -0.3801
....Labels  4.3452 5.0922 6.5510 3.2017
....Loss: -0.3991157114505768
Finished charting dependencies. Data is not getting mixed between instances in the same batch.


Debug session complete. No errors detected.


In [40]:
# this is a buggy model. Can you spot the bugs?

class BuggyGraphNet(nn.Module):
    
    def __init__(self, input_dim, output_dim, capacity):

        super(BuggyGraphNet,self).__init__()
        self.layers = nn.ModuleList()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.n_hidden = capacity
        unit_sequence = utils.unit_sequence(self.input_dim, 
                                            self.output_dim, 
                                            self.n_hidden)
        self.node_projector = nn.Linear(N_FEATURES_, PROJECTOR_DIM)
        self.relu = nn.ReLU()
        # set up hidden layers
        for ind,n_units in enumerate(unit_sequence[:-2]):
            size_out_ = unit_sequence[ind+1]
            layer = nn.Linear(n_units, size_out_)
            self.layers.append(layer)

        # set up output layer
        size_in_ = unit_sequence[-2]
        size_out_ = unit_sequence[-1]
        layer = nn.Linear(size_in_, size_out_)
        self.layers.append(layer)
    
    def forward(self, data):
        x = data.x
        x = x.view(data.num_graphs, MAX_N_ATOMS, N_FEATURES_)  #this mixes information between instances because the 2D feature, matrices of different polymers. Stacked along the zero dimension in line
        x = self.node_projector(x)
        x_mean = x.mean(dim=0) # Spoiler! this is the bug.  # #  mean to be taken over the zeroth  Dimension.
        x = x - x_mean[None, :, :] # make use of broadcasting
        x = x.sum(dim=1)
        for i in range(len(self.layers)):
            x = self.layers[i](x)
            if i < (self.n_hidden - 1):
                x = self.relu(x)
   
        return x.view(data.num_graphs,)

# a list of models that are bug free!
capacity_ls = [1,2,3]
buggy_graphnet_class_ls = [lambda : BuggyGraphNet(PROJECTOR_DIM, 1, capacity) for capacity in
                          capacity_ls]

In [41]:
# this test should not pass since we are using a buggy model

ds = dl_debug.DebugSession(buggy_graphnet_class_ls, 'gnn', capacity_ls, data_set_, zero_data_set, loss_fn,
                 device, do_chart_dependencies=True)
best_model_capacity = ds.main()

Training data contains 4 points


Beginning to chart dependencies
..Epoch 0
....Outputs -0.1933 -0.1732 -0.1948 -0.1743
....Labels  4.3452 5.0922 6.5510 3.2017
....Loss: -0.19329825043678284


ValueError: Data is getting mixed between instances in the same batch.

# Overfit training data
The capacity of your architecture should be just large enough to overfit the training data. 

In [42]:
# this cell should return the integer 1
# find the minimum capacity, my net over fit the entire training data. 
# for this data set training procedure and architecture. That the optimal capacity is one. Therefore we need only one hidden layer to completely over fit the training data, but this may not be the case on different architectures in different data sets.

ds = dl_debug.DebugSession(correct_model_class_ls, model_type, capacity_ls, data_set, zero_data_set, loss_fn,
                 device, do_choose_model_size_by_overfit=True, trainer=trainer)
ds.main()

Training data contains 676 points


Beginning model size search

..Training model 0 


....Epoch 0
......[rmse] 4.612588712779379 [r2] -7.962118969080686
......Outputs -0.0134 0.0056 0.0020 0.0056 -0.0062 0.0249 0.0047 0.0048 0.0136 -0.0035
......Labels  5.0497 5.6991 2.9694 5.7012 5.3739 2.8673 6.7030 5.1590 2.1188 3.1085
......[best rmse] 4.612588712779379 [best r2] -7.962118969080686

....Epoch 1
......[rmse] 3.7001437196337448 [r2] -4.767112549579306
......Outputs 0.4713 0.6923 0.5647 0.6923 0.6993 0.9451 0.5744 0.6923 0.5705 0.7246
......Labels  5.0497 5.6991 2.9694 5.7012 5.3739 2.8673 6.7030 5.1590 2.1188 3.1085
......[best rmse] 3.7001437196337448 [best r2] -4.767112549579306

....Epoch 2
......[rmse] 2.6620075555319445 [r2] -1.9849688850561997
......Outputs 2.4214 3.8174 2.9446 3.8174 3.8828 5.0795 3.0282 3.7635 2.8393 4.1709
......Labels  5.0497 5.6991 2.9694 5.7012 5.3739 2.8673 6.7030 5.1590 2.1188 3.1085
......[best rmse] 2.6620075555319445 [best r2] -1.9849688850561997

.


....Epoch 27
......[rmse] 0.18624584283360338 [r2] 0.9853885129906151
......Outputs 5.0278 5.7117 2.8758 5.7117 5.4681 2.7502 6.6846 5.1434 2.0905 3.2711
......Labels  5.0497 5.6991 2.9694 5.7012 5.3739 2.8673 6.7030 5.1590 2.1188 3.1085
......[best rmse] 0.18624584283360338 [best r2] 0.9853885129906151

....Epoch 28
......[rmse] 0.1770083492139858 [r2] 0.9868019811415891
......Outputs 5.0403 5.7204 2.8784 5.7204 5.4566 2.7812 6.6953 5.1520 2.0945 3.2734
......Labels  5.0497 5.6991 2.9694 5.7012 5.3739 2.8673 6.7030 5.1590 2.1188 3.1085
......[best rmse] 0.1770083492139858 [best r2] 0.9868019811415891

....Epoch 29
......[rmse] 0.16851980051630988 [r2] 0.9880374679705571
......Outputs 5.0411 5.7291 2.8729 5.7291 5.4359 2.8053 6.7010 5.1464 2.0989 3.2685
......Labels  5.0497 5.6991 2.9694 5.7012 5.3739 2.8673 6.7030 5.1590 2.1188 3.1085
......[best rmse] 0.16851980051630988 [best r2] 0.9880374679705571

....Epoch 30
......[rmse] 0.16083067376418106 [r2] 0.9891042029938654
......Outputs


....Epoch 54
......[rmse] 0.10203932901514434 [r2] 0.9956141231895166
......Outputs 4.9797 5.6025 2.9214 5.6025 5.3354 2.7861 6.5456 5.0430 2.0795 3.0744
......Labels  5.0497 5.6991 2.9694 5.7012 5.3739 2.8673 6.7030 5.1590 2.1188 3.1085
......[best rmse] 0.10203932901514434 [best r2] 0.9956141231895166

....Epoch 55
......[rmse] 0.0878155713576997 [r2] 0.9967516388813022
......Outputs 5.0818 5.9089 3.0058 5.9089 5.4507 2.8753 6.8126 5.2135 2.1481 3.2101
......Labels  5.0497 5.6991 2.9694 5.7012 5.3739 2.8673 6.7030 5.1590 2.1188 3.1085
......[best rmse] 0.0878155713576997 [best r2] 0.9967516388813022

....Epoch 56
......[rmse] 0.07529761349123075 [r2] 0.9976117286745693
......Outputs 5.0626 5.7334 2.9925 5.7334 5.3914 2.8787 6.7178 5.1694 2.1361 3.2003
......Labels  5.0497 5.6991 2.9694 5.7012 5.3739 2.8673 6.7030 5.1590 2.1188 3.1085
......[best rmse] 0.07529761349123075 [best r2] 0.9976117286745693

....Epoch 57
......[rmse] 0.07170390378267145 [r2] 0.9978342573849696
......Outputs


....Epoch 81
......[rmse] 0.13786100175705793 [r2] 0.9919942117176782
......Outputs 4.9311 5.7153 2.8726 5.7153 5.2398 2.8219 6.6626 5.1022 2.0454 3.1014
......Labels  5.0497 5.6991 2.9694 5.7012 5.3739 2.8673 6.7030 5.1590 2.1188 3.1085
......[best rmse] 0.05041559512505494 [best r2] 0.9989293413325728

....Epoch 82
......[rmse] 0.18659165369792868 [r2] 0.985334203033805
......Outputs 5.2940 6.0038 3.0816 6.0038 5.5806 2.9867 7.0116 5.4437 2.1917 3.1974
......Labels  5.0497 5.6991 2.9694 5.7012 5.3739 2.8673 6.7030 5.1590 2.1188 3.1085
......[best rmse] 0.05041559512505494 [best r2] 0.9989293413325728

....Epoch 83
......[rmse] 0.21130637087627818 [r2] 0.9811918351220749
......Outputs 5.2646 5.7970 3.1453 5.7970 5.6306 2.9626 6.8148 5.2921 2.2496 3.1620
......Labels  5.0497 5.6991 2.9694 5.7012 5.3739 2.8673 6.7030 5.1590 2.1188 3.1085
......[best rmse] 0.05041559512505494 [best r2] 0.9989293413325728

....Epoch 84
......[rmse] 0.2519654451628788 [r2] 0.9732574256877856
......Outputs


....Epoch 8
......[rmse] 1.1358440886156123 [r2] 0.4565511361563186
......Outputs 3.5278 5.8883 2.3460 5.8883 6.2576 3.4048 6.7548 5.1609 2.5861 2.5665
......Labels  5.0497 5.6991 2.9694 5.7012 5.3739 2.8673 6.7030 5.1590 2.1188 3.1085
......[best rmse] 1.1358440886156123 [best r2] 0.4565511361563186

....Epoch 9
......[rmse] 1.0209814300624607 [r2] 0.5609065820719996
......Outputs 4.3393 6.2014 2.8961 6.2014 6.5363 3.7308 7.6351 5.9285 2.9399 3.1642
......Labels  5.0497 5.6991 2.9694 5.7012 5.3739 2.8673 6.7030 5.1590 2.1188 3.1085
......[best rmse] 1.0209814300624607 [best r2] 0.5609065820719996

....Epoch 10
......[rmse] 0.8998264030704474 [r2] 0.6589337820374406
......Outputs 4.4791 5.5616 2.9345 5.5616 5.8141 3.3497 7.2552 5.5819 2.7109 3.1355
......Labels  5.0497 5.6991 2.9694 5.7012 5.3739 2.8673 6.7030 5.1590 2.1188 3.1085
......[best rmse] 0.8998264030704474 [best r2] 0.6589337820374406

....Epoch 11
......[rmse] 0.7903523235296026 [r2] 0.7368746386053098
......Outputs 4.8839


....Epoch 35
......[rmse] 0.12425842724798561 [r2] 0.9934961136206648
......Outputs 5.0425 5.7292 2.8959 5.7292 5.3878 2.8809 6.7036 5.1413 1.9731 3.1354
......Labels  5.0497 5.6991 2.9694 5.7012 5.3739 2.8673 6.7030 5.1590 2.1188 3.1085
......[best rmse] 0.12425842724798561 [best r2] 0.9934961136206648

....Epoch 36
......[rmse] 0.11925680312291644 [r2] 0.9940091621190668
......Outputs 5.0153 5.7113 2.8837 5.7113 5.3555 2.8689 6.6773 5.1159 1.9778 3.1235
......Labels  5.0497 5.6991 2.9694 5.7012 5.3739 2.8673 6.7030 5.1590 2.1188 3.1085
......[best rmse] 0.11925680312291644 [best r2] 0.9940091621190668

....Epoch 37
......[rmse] 0.11619940093482997 [r2] 0.9943124003477559
......Outputs 4.9986 5.7106 2.8768 5.7106 5.3297 2.8609 6.6729 5.1147 1.9828 3.1162
......Labels  5.0497 5.6991 2.9694 5.7012 5.3739 2.8673 6.7030 5.1590 2.1188 3.1085
......[best rmse] 0.11619940093482997 [best r2] 0.9943124003477559

....Epoch 38
......[rmse] 0.11400678607364434 [r2] 0.994525018620513
......Output


....Epoch 62
......[rmse] 0.04696316101055017 [r2] 0.9990709568563683
......Outputs 5.0308 5.7186 2.9562 5.7186 5.3661 2.8561 6.6816 5.1415 2.1062 3.1067
......Labels  5.0497 5.6991 2.9694 5.7012 5.3739 2.8673 6.7030 5.1590 2.1188 3.1085
......[best rmse] 0.04696316101055017 [best r2] 0.9990709568563683

....Epoch 63
......[rmse] 0.04627569783207959 [r2] 0.9990979570968368
......Outputs 5.0368 5.7410 2.9566 5.7410 5.3719 2.8653 6.7034 5.1522 2.1078 3.1151
......Labels  5.0497 5.6991 2.9694 5.7012 5.3739 2.8673 6.7030 5.1590 2.1188 3.1085
......[best rmse] 0.04627569783207959 [best r2] 0.9990979570968368

....Epoch 64
......[rmse] 0.04749614184493341 [r2] 0.9990497499377753
......Outputs 5.0669 5.7569 2.9720 5.7569 5.3926 2.8807 6.7305 5.1741 2.1151 3.1229
......Labels  5.0497 5.6991 2.9694 5.7012 5.3739 2.8673 6.7030 5.1590 2.1188 3.1085
......[best rmse] 0.04627569783207959 [best r2] 0.9990979570968368

....Epoch 65
......[rmse] 0.047006401041857264 [r2] 0.9990692452872442
......Outp


....Epoch 89
......[rmse] 0.06733733621037234 [r2] 0.9980900011522416
......Outputs 5.0519 5.6262 2.9650 5.6262 5.3559 2.8803 6.6719 5.1513 2.1259 3.1243
......Labels  5.0497 5.6991 2.9694 5.7012 5.3739 2.8673 6.7030 5.1590 2.1188 3.1085
......[best rmse] 0.04627569783207959 [best r2] 0.9990979570968368

....Epoch 90
......[rmse] 0.06346360983383428 [r2] 0.9983034339054966
......Outputs 4.9871 5.6280 2.9195 5.6280 5.3222 2.8196 6.5987 5.1061 2.0993 3.0420
......Labels  5.0497 5.6991 2.9694 5.7012 5.3739 2.8673 6.7030 5.1590 2.1188 3.1085
......[best rmse] 0.04627569783207959 [best r2] 0.9990979570968368

....Epoch 91
......[rmse] 0.05468207082057125 [r2] 0.9987404623703117
......Outputs 5.0773 5.8316 2.9765 5.8316 5.4029 2.8702 6.7564 5.2076 2.1287 3.1186
......Labels  5.0497 5.6991 2.9694 5.7012 5.3739 2.8673 6.7030 5.1590 2.1188 3.1085
......[best rmse] 0.04627569783207959 [best r2] 0.9990979570968368

....Epoch 92
......[rmse] 0.05025693958774749 [r2] 0.9989360693557428
......Outpu

1

# Run all tests

In [43]:
ds = dl_debug.DebugSession(correct_model_class_ls, model_type, capacity_ls, data_set, zero_data_set, loss_fn,
                 device, do_all_tests=True, trainer=trainer) # do_all_tests=True test all together
ds.main()

Training data contains 676 points


Verified that shape of model predictions is equal to shape of labels


Checking input-independent baseline
..last epoch real_data_loss 0.23777171038091183
..last epoch zero_data_loss 14.244109153747559
Input-independent baseline is verified


Checking if a small batch can be overfit
..Epoch 0
....Outputs -0.1052 -0.1486 -0.1483 -0.1343 -0.1052
....Labels  5.7012 2.9694 5.3739 5.0497 5.6991
....Loss: 5.186427729446679
....R2: -24.679339335586373
..Epoch 1
....Outputs -0.0979 -0.0846 -0.0802 -0.0420 -0.0420
....Labels  2.9694 5.0497 5.3739 5.6991 5.7012
....Loss: 5.127613424741939
....R2: -24.10023388046654
..Epoch 2
....Outputs -0.0176 -0.0377 0.0243 -0.0491 0.0243
....Labels  5.3739 5.0497 5.6991 2.9694 5.7012
....Loss: 5.069319288314261
....R2: -23.53276465049816
..Epoch 3
....Outputs 0.0135 0.0490 0.1007 0.0027 0.1007
....Labels  5.0497 5.3739 5.7012 2.9694 5.6991
....Loss: 5.004514944559789
....R2: -22.909536406766325
..Epoch 4
....Outputs 0.1922 


Beginning to chart dependencies
..Epoch 0
....Outputs -0.0734 -0.0912 -0.0731 -0.0912 -0.0706
....Labels  5.0497 5.6991 2.9694 5.7012 5.3739
....Loss: -0.07340135425329208
Finished charting dependencies. Data is not getting mixed between instances in the same batch.


Beginning model size search

..Training model 0 


....Epoch 0
......[rmse] 4.697626745953917 [r2] -8.295617695251963
......Outputs -0.1117 -0.1060 -0.1165 -0.1060 -0.1006 -0.0928 -0.0947 -0.1026 -0.0865 -0.0975
......Labels  5.0497 5.6991 2.9694 5.7012 5.3739 2.8673 6.7030 5.1590 2.1188 3.1085
......[best rmse] 4.697626745953917 [best r2] -8.295617695251963

....Epoch 1
......[rmse] 3.79299482316163 [r2] -5.060183027811168
......Outputs 0.3317 0.6008 0.4492 0.6008 0.6427 0.7694 0.4853 0.5329 0.4608 0.6642
......Labels  5.0497 5.6991 2.9694 5.7012 5.3739 2.8673 6.7030 5.1590 2.1188 3.1085
......[best rmse] 3.79299482316163 [best r2] -5.060183027811168

....Epoch 2
......[rmse] 2.6929199222363396 [r2] -2.0546968661917835



....Epoch 26
......[rmse] 0.17978612734091132 [r2] 0.986384499947238
......Outputs 5.0575 5.7334 2.9197 5.7334 5.4801 2.8086 6.6849 5.1511 1.9417 3.1805
......Labels  5.0497 5.6991 2.9694 5.7012 5.3739 2.8673 6.7030 5.1590 2.1188 3.1085
......[best rmse] 0.17978612734091132 [best r2] 0.986384499947238

....Epoch 27
......[rmse] 0.16877867505441155 [r2] 0.9880006868499874
......Outputs 5.0523 5.7444 2.9190 5.7444 5.4616 2.8246 6.6918 5.1436 1.9713 3.1676
......Labels  5.0497 5.6991 2.9694 5.7012 5.3739 2.8673 6.7030 5.1590 2.1188 3.1085
......[best rmse] 0.16877867505441155 [best r2] 0.9880006868499874

....Epoch 28
......[rmse] 0.15896543909911795 [r2] 0.9893554656228891
......Outputs 5.0457 5.7302 2.9209 5.7302 5.4496 2.8381 6.7141 5.1486 2.0046 3.1647
......Labels  5.0497 5.6991 2.9694 5.7012 5.3739 2.8673 6.7030 5.1590 2.1188 3.1085
......[best rmse] 0.15896543909911795 [best r2] 0.9893554656228891

....Epoch 29
......[rmse] 0.15002232654169556 [r2] 0.9905194609090558
......Outputs


....Epoch 53
......[rmse] 0.10398236531171703 [r2] 0.9954455008506692
......Outputs 4.9570 5.7174 2.9241 5.7174 5.3295 2.7719 6.5874 4.9980 2.1040 2.9676
......Labels  5.0497 5.6991 2.9694 5.7012 5.3739 2.8673 6.7030 5.1590 2.1188 3.1085
......[best rmse] 0.08795076050275073 [best r2] 0.9967416296954872

....Epoch 54
......[rmse] 0.08808692825410958 [r2] 0.9967315324906999
......Outputs 5.0766 5.9677 3.0114 5.9677 5.4807 2.8979 6.8377 5.2659 2.1485 3.2064
......Labels  5.0497 5.6991 2.9694 5.7012 5.3739 2.8673 6.7030 5.1590 2.1188 3.1085
......[best rmse] 0.08795076050275073 [best r2] 0.9967416296954872

....Epoch 55
......[rmse] 0.0714049693617013 [r2] 0.9978522777559701
......Outputs 5.0549 5.6105 2.9938 5.6105 5.3945 2.8845 6.6984 5.1662 2.1266 3.1303
......Labels  5.0497 5.6991 2.9694 5.7012 5.3739 2.8673 6.7030 5.1590 2.1188 3.1085
......[best rmse] 0.0714049693617013 [best r2] 0.9978522777559701

....Epoch 56
......[rmse] 0.06634434034065682 [r2] 0.9981459177278036
......Outputs


....Epoch 80
......[rmse] 0.06672539731476879 [r2] 0.9981245582585972
......Outputs 5.1240 5.8061 3.0038 5.8061 5.4303 2.9085 6.7884 5.2464 2.1389 3.1207
......Labels  5.0497 5.6991 2.9694 5.7012 5.3739 2.8673 6.7030 5.1590 2.1188 3.1085
......[best rmse] 0.04583921155596092 [best r2] 0.9991148935226217

....Epoch 81
......[rmse] 0.07101952810699096 [r2] 0.9978754018183911
......Outputs 5.0981 5.7357 3.0082 5.7357 5.4180 2.8845 6.7259 5.1611 2.1507 3.0961
......Labels  5.0497 5.6991 2.9694 5.7012 5.3739 2.8673 6.7030 5.1590 2.1188 3.1085
......[best rmse] 0.04583921155596092 [best r2] 0.9991148935226217

....Epoch 82
......[rmse] 0.08562312864069073 [r2] 0.9969118141664036
......Outputs 4.9375 5.6237 2.9234 5.6237 5.2959 2.8051 6.5740 5.0219 2.0914 3.0701
......Labels  5.0497 5.6991 2.9694 5.7012 5.3739 2.8673 6.7030 5.1590 2.1188 3.1085
......[best rmse] 0.04583921155596092 [best r2] 0.9991148935226217

....Epoch 83
......[rmse] 0.10017967138776594 [r2] 0.9957725308558085
......Outpu


....Epoch 7
......[rmse] 1.2129663636464343 [r2] 0.3802468306217779
......Outputs 3.1858 5.9666 2.2142 5.9666 6.4575 3.1544 6.6191 4.7816 2.5952 2.4258
......Labels  5.0497 5.6991 2.9694 5.7012 5.3739 2.8673 6.7030 5.1590 2.1188 3.1085
......[best rmse] 1.2129663636464343 [best r2] 0.3802468306217779

....Epoch 8
......[rmse] 1.0471001272744604 [r2] 0.5381534905716436
......Outputs 3.6408 5.8703 2.5117 5.8703 6.3322 3.2693 7.0701 5.2588 2.8016 2.9213
......Labels  5.0497 5.6991 2.9694 5.7012 5.3739 2.8673 6.7030 5.1590 2.1188 3.1085
......[best rmse] 1.0471001272744604 [best r2] 0.5381534905716436

....Epoch 9
......[rmse] 0.9251318971703217 [r2] 0.6394806735957446
......Outputs 4.3066 6.0685 2.9563 6.0685 6.4540 3.5286 7.7239 5.8273 3.0526 3.4884
......Labels  5.0497 5.6991 2.9694 5.7012 5.3739 2.8673 6.7030 5.1590 2.1188 3.1085
......[best rmse] 0.9251318971703217 [best r2] 0.6394806735957446

....Epoch 10
......[rmse] 0.8102487751575443 [r2] 0.7234599689633869
......Outputs 4.3155 


....Epoch 34
......[rmse] 0.12492317200126112 [r2] 0.993426339860534
......Outputs 5.0295 5.7349 2.9282 5.7349 5.3966 2.8679 6.7137 5.1607 2.0943 3.1547
......Labels  5.0497 5.6991 2.9694 5.7012 5.3739 2.8673 6.7030 5.1590 2.1188 3.1085
......[best rmse] 0.12492317200126112 [best r2] 0.993426339860534

....Epoch 35
......[rmse] 0.11935246036327472 [r2] 0.9939995476260416
......Outputs 5.0265 5.7356 2.9319 5.7356 5.3960 2.8738 6.7063 5.1554 2.1000 3.1536
......Labels  5.0497 5.6991 2.9694 5.7012 5.3739 2.8673 6.7030 5.1590 2.1188 3.1085
......[best rmse] 0.11935246036327472 [best r2] 0.9939995476260416

....Epoch 36
......[rmse] 0.11396668879361449 [r2] 0.9945288691501765
......Outputs 5.0247 5.7284 2.9347 5.7284 5.3875 2.8690 6.7016 5.1530 2.1057 3.1498
......Labels  5.0497 5.6991 2.9694 5.7012 5.3739 2.8673 6.7030 5.1590 2.1188 3.1085
......[best rmse] 0.11396668879361449 [best r2] 0.9945288691501765

....Epoch 37
......[rmse] 0.10899873405341555 [r2] 0.9949954601850939
......Outputs


....Epoch 61
......[rmse] 0.06945237300948703 [r2] 0.9979681323352911
......Outputs 5.1129 5.8481 2.9995 5.8481 5.4457 2.9368 6.7978 5.1972 2.1405 3.1886
......Labels  5.0497 5.6991 2.9694 5.7012 5.3739 2.8673 6.7030 5.1590 2.1188 3.1085
......[best rmse] 0.06790388537142995 [best r2] 0.9980577260277141

....Epoch 62
......[rmse] 0.056471738216607184 [r2] 0.9986566674066664
......Outputs 5.0305 5.6278 2.9479 5.6278 5.3705 2.8470 6.6703 5.1289 2.1066 3.1118
......Labels  5.0497 5.6991 2.9694 5.7012 5.3739 2.8673 6.7030 5.1590 2.1188 3.1085
......[best rmse] 0.056471738216607184 [best r2] 0.9986566674066664

....Epoch 63
......[rmse] 0.05485249378654779 [r2] 0.9987325991481423
......Outputs 5.0049 5.7034 2.9475 5.7034 5.3479 2.8276 6.6555 5.1449 2.1002 3.0903
......Labels  5.0497 5.6991 2.9694 5.7012 5.3739 2.8673 6.7030 5.1590 2.1188 3.1085
......[best rmse] 0.05485249378654779 [best r2] 0.9987325991481423

....Epoch 64
......[rmse] 0.0535341345515509 [r2] 0.9987927900309652
......Outp


....Epoch 88
......[rmse] 0.19646339947174335 [r2] 0.9837413474614054
......Outputs 5.2399 6.0870 3.0383 6.0870 5.5424 3.0950 7.0171 5.3427 2.1653 3.2087
......Labels  5.0497 5.6991 2.9694 5.7012 5.3739 2.8673 6.7030 5.1590 2.1188 3.1085
......[best rmse] 0.0470499863146051 [best r2] 0.9990675184583666

....Epoch 89
......[rmse] 0.23125619843196377 [r2] 0.9774727590079835
......Outputs 5.3376 5.8591 3.1420 5.8591 5.6503 3.0346 6.9148 5.3500 2.2539 3.1560
......Labels  5.0497 5.6991 2.9694 5.7012 5.3739 2.8673 6.7030 5.1590 2.1188 3.1085
......[best rmse] 0.0470499863146051 [best r2] 0.9990675184583666

....Epoch 90
......[rmse] 0.2531895323019975 [r2] 0.9729969553864217
......Outputs 4.7817 5.2728 2.8842 5.2728 5.2024 2.5673 6.2821 4.9188 2.0786 2.9722
......Labels  5.0497 5.6991 2.9694 5.7012 5.3739 2.8673 6.7030 5.1590 2.1188 3.1085
......[best rmse] 0.0470499863146051 [best r2] 0.9990675184583666

....Epoch 91
......[rmse] 0.2978939957641482 [r2] 0.9626195314719013
......Outputs 4.

1